In [41]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import datetime
import yfinance as yf
import plotly.express as px
from statsmodels.tsa.stattools import adfuller
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [42]:
# =========================
# Step 1: Data Preparation
# =========================
start = datetime.datetime(1990, 1, 1)
end   = datetime.datetime(2026, 1, 31)

fred_codes = [
    'DGS10', 'T10Y2Y', 'DGS5', 'DGS2', 'TB3MS',
    'BAA', 'AAA',
    'CPIAUCSL', 'UNRATE', 'INDPRO',
    'USREC', 'VIXCLS'
]

In [43]:
# 1) Pull FRED (daily/mixed) and convert to monthly (month-start index, end-of-month value)
df_fred = pdr.DataReader(fred_codes, 'fred', start, end)

# df_fred_m = df_fred.resample('MS').last() # 统一成月频

In [44]:
# df_fred.head()

In [45]:
df_fred_m = df_fred.resample('MS').last() # 统一成月频

In [46]:
# df_fred_m.head()

In [47]:
# 2) Pull MOVE and convert to monthly
df_move = yf.download("^MOVE", start="1990-01-01", end="2026-01-31", progress=False)

In [48]:
# df_move.head()

In [49]:
df_move_m = df_move['Close'].resample('MS').last() # 同样月频化

In [50]:
# df_move_m.head()

In [51]:
# 3) Merge (monthly)
df = df_fred_m.merge(df_move_m, left_on="DATE", right_on="Date", how="outer").set_index(df_fred_m.index)

In [52]:
# df.head()

In [53]:
# df.tail()

In [54]:
# 'CPIAUCSL', 'UNRATE', 'INDPRO' 都缺一个值
# Fill key monthly macro gaps (safe forward-fill for monthly series)
df[['CPIAUCSL', 'UNRATE', 'INDPRO']] = df[['CPIAUCSL', 'UNRATE', 'INDPRO']].ffill()
# CPI/失业率/工业生产有时是月度发布但会缺值，用前值填充

In [55]:
# df.describe()
# 现在除了Move都是433个值

In [56]:
# =========================
# Step 1b: Construct target y (10Y bond monthly excess return)
# =========================
duration = 9  # approx duration for 10Y Treasury

# price return from yield change
df['price_ret'] = -duration * (df['DGS10'].diff() / 100)

# carry return (use yield at start of month => shift(1))
df['carry_ret'] = (df['DGS10'].shift(1) / 100) / 12

df['total_bond_ret'] = df['price_ret'] + df['carry_ret']
df['excess_return'] = df['total_bond_ret'] - ((df['TB3MS'] / 100) / 12)

In [57]:
# =========================
# Step 1c: Construct predictors X
# =========================
df['slope'] = df['T10Y2Y']
df['curvature'] = (2 * df['DGS5']) - df['DGS2'] - df['DGS10']
df['short_rate'] = df['TB3MS']
df['default_spread'] = df['BAA'] - df['AAA']
df['inflation_yoy'] = df['CPIAUCSL'].pct_change(12)
df['unemployment'] = df['UNRATE']
df['recession_indicator'] = df['USREC']
df['industrial_prod_yoy'] = df['INDPRO'].pct_change(12)
df['vix_index'] = df['VIXCLS']

In [58]:
# Bond volatility: use MOVE when available; otherwise use realized vol from daily DGS10
# (This part uses daily DGS10, so compute from df_fred daily then monthly last)
df_fred['DGS10_filled'] = df_fred['DGS10'].ffill()
df_fred['yield_diff'] = df_fred['DGS10_filled'].diff()
df_fred['realized_daily_vol_30d'] = df_fred['yield_diff'].rolling(window=30).std() * np.sqrt(252)  # annualized
realized_m = df_fred['realized_daily_vol_30d'].resample('MS').last() * 100  # to % scale
df['bond_volatility'] = df['^MOVE'].fillna(realized_m)

# Bond momentum: past 12m excess return, lagged by 1 month
df['excess_return_12m'] = df['excess_return'].rolling(window=12).sum()
df['bond_momentum'] = df['excess_return_12m'].shift(1) # shift(1)：用“到上个月为止”的动量来预测本月，避免偷看本月回报

In [59]:
# =========================
# Step 1d: Lag predictors by 1 month (avoid look-ahead bias)
# =========================
base_predictors = [
    'slope', 'curvature', 'short_rate', 'default_spread', 'inflation_yoy',
    'unemployment', 'recession_indicator', 'industrial_prod_yoy',
    'vix_index', 'bond_volatility', 'bond_momentum'
]

X = df[base_predictors].shift(1)
y = df['excess_return']

# X.info()
# y.info()

In [60]:
df_model = pd.concat([y, X], axis=1).dropna()
df_model.rename(columns={'excess_return': 'y'}, inplace=True)

# =========================
# (Optional but common) Make short_rate stationary + add AR(1) term
# =========================
df_model['short_rate_diff'] = df_model['short_rate'].diff()
df_model['y_lag1'] = df_model['y'].shift(1)

feature_cols = [
    'slope', 'curvature', 'short_rate_diff', 'default_spread', 'inflation_yoy',
    'unemployment', 'recession_indicator', 'industrial_prod_yoy',
    'vix_index', 'bond_volatility', 'bond_momentum', 'y_lag1'
]

df_model = df_model[['y'] + feature_cols].dropna()

In [61]:
# combine and drop NaN due to shift/pct_change
# 拼成一个表：第一列是 y，后面是滞后后的 X
# df_final = pd.concat([y, X], axis=1).dropna() # dropna()：由于 diff/pct_change/rolling/shift 会引入 NaN，统一删除

# df_final.head()

In [62]:
# df_final.info()

In [63]:
def run_stationarity_test(df):
    results = []
    for col in df.columns:
        # perform Augmented Dickey-Fuller test
        adf_result = adfuller(df[col].dropna())

        # extract values
        results.append({
            'Variable': col,
            'ADF Statistic': round(adf_result[0], 4),
            'p-value': round(adf_result[1], 4),
            'Stationary (5%)': 'Yes' if adf_result[1] < 0.05 else 'No'
        })

    return pd.DataFrame(results)

In [64]:
# stationarity_table = run_stationarity_test(df_final)
# stationarity_table # 输出表格检查哪些变量不平稳

We found that the `short_rate` is non-stationary. Thus we need to make it stationary by taking the first difference.

In [65]:
# make short-rate stationary: create the first difference for the short rate
# df_final['short_rate_diff'] = df_final['short_rate'].diff() # diff()：短端利率做一阶差分常常更平稳
# df_final.dropna(inplace=True) # dropna：第一行差分必然 NaN，删掉

In [66]:
# stationarity_table = run_stationarity_test(df_final)
# stationarity_table

In [67]:
# ============================================================
# Step 3: Regularised Regression (Ridge & Lasso) + Proper TS-CV
# Step 5: Statistical Evaluation (OOS R^2 vs mean benchmark)
# Step 6: Economic Evaluation (Market timing: Sharpe, CumRet)
# ============================================================

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error

In [68]:
# ---------- helpers ----------
# 定义r2函数
def oos_r2_vs_benchmark(y_true, y_pred, y_bench):
    """OOS R^2 = 1 - SSE_model / SSE_benchmark"""
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    y_bench = np.asarray(y_bench, dtype=float)
    sse_model = np.sum((y_true - y_pred) ** 2)
    sse_bench = np.sum((y_true - y_bench) ** 2)
    return np.nan if sse_bench == 0 else 1 - sse_model / sse_bench

# 年化Sharpe
def ann_sharpe(r, periods_per_year=12):
    r = np.asarray(r, dtype=float)
    mu = np.nanmean(r)
    sd = np.nanstd(r, ddof=1)
    return np.nan if sd == 0 else (mu / sd) * np.sqrt(periods_per_year)

In [69]:
# ---------- data ----------
# index = monthly dates, columns = ['y'] + feature columns
y = df_model["y"].copy() # y是目标：当月超额收益
X = df_model.drop(columns=["y"]).copy() # x是所有特征
feature_names = X.columns.tolist()

In [70]:
# ---------- walk-forward settings ----------
initial_train = 120     # 10 years (monthly)：初始训练窗口120月（10年）
horizon = 1             # 1-step ahead：预测1个月ahead
if len(df_model) <= initial_train + horizon:
    raise ValueError("Not enough observations for the chosen initial_train/horizon.") # 样本不够就直接报错

# Inner CV (ONLY on training set each step)
inner_tscv = TimeSeriesSplit(n_splits=5) # 把训练集按时间切5次：每次验证集都在训练集之后，不会发生“用未来训练过去”

# alpha grids (lambda candidates)
ridge_grid = {"model__alpha": np.logspace(-4, 4, 25)} # np.logspace(-4,4,25) 生成 1e-4 到 1e4 的 25 个对数间隔点
lasso_grid = {"model__alpha": np.logspace(-4, 1, 25)} # lasso often needs smaller range

# pipelines (scale inside pipeline => no leakage)
ridge_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Ridge())
])

# 每次fit时，1.scaler只在训练集上估计均值/标准差，2.把训练集标准化，3.用标准化后的数据训练Ridge

lasso_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", Lasso(max_iter=10000)) # Lasso同理，只是加了max_iter=10000避免迭代不收敛
])

In [71]:
# 储存结果的list
# ---------- walk-forward OOS loop ----------
oos_dates = []
y_true_list = []
y_bench_list = []
yhat_ridge_list = []
yhat_lasso_list = []
best_alpha_ridge = []
best_alpha_lasso = []

# For economic evaluation: store positions (sign of forecast) and realized returns
pos_ridge = []
pos_lasso = []

# Lasso selection frequency (count non-zero across OOS windows)
lasso_nonzero_counts = pd.Series(0, index=feature_names, dtype=int)
# 给每个特征一个计数器，初始0，每个月如果该特征Lasso系数非零，就+1

In [72]:
for t in range(initial_train, len(df_model) - (horizon - 1)): # 从第120个点开始做OOS预测，一直预测到最后
    train_end = t
    test_idx = t  # predict month t using data up to t-1

    X_train = X.iloc[:train_end]
    y_train = y.iloc[:train_end]
    X_test  = X.iloc[test_idx:test_idx + 1]
    y_test  = y.iloc[test_idx:test_idx + 1] # expanding window

    # benchmark: expanding mean，benchmark就是历史均值预测（只用训练集均值）
    bench_forecast = float(y_train.mean())

    # Ridge：在训练集里做 TS-CV 选 alpha
    # -------- Ridge with TS-CV --------
    ridge_gs = GridSearchCV(
        estimator=ridge_pipe,
        param_grid=ridge_grid,
        cv=inner_tscv,
        scoring="neg_mean_squared_error",
        n_jobs=-1
    )
    ridge_gs.fit(X_train, y_train)
    yhat_ridge = float(ridge_gs.best_estimator_.predict(X_test)[0])

    # Lasso 同样逻辑 + 记录变量选择
    # -------- Lasso with TS-CV --------
    lasso_gs = GridSearchCV(
        estimator=lasso_pipe,
        param_grid=lasso_grid,
        cv=inner_tscv,
        scoring="neg_mean_squared_error",
        n_jobs=-1
    )
    lasso_gs.fit(X_train, y_train)
    best_lasso_est = lasso_gs.best_estimator_
    yhat_lasso = float(best_lasso_est.predict(X_test)[0])

    # record lasso selection (non-zero coefs on standardized scale)
    coefs = best_lasso_est.named_steps["model"].coef_
    lasso_nonzero_counts += (np.abs(coefs) > 1e-12).astype(int)

    # store
    oos_dates.append(df_model.index[test_idx])
    y_true_list.append(float(y_test.values[0]))
    y_bench_list.append(bench_forecast)

    yhat_ridge_list.append(yhat_ridge)
    yhat_lasso_list.append(yhat_lasso)

    best_alpha_ridge.append(ridge_gs.best_params_["model__alpha"])
    best_alpha_lasso.append(lasso_gs.best_params_["model__alpha"])

    # simple market timing positions (sign of forecast)
    pos_ridge.append(np.sign(yhat_ridge))
    pos_lasso.append(np.sign(yhat_lasso))

In [73]:
# 把list合成results DataFrame
# ---------- collect results ----------
results = pd.DataFrame({
    "y_true": y_true_list,
    "y_bench_mean": y_bench_list,
    "yhat_ridge": yhat_ridge_list,
    "yhat_lasso": yhat_lasso_list,
    "best_alpha_ridge": best_alpha_ridge,
    "best_alpha_lasso": best_alpha_lasso,
    "pos_ridge": pos_ridge,
    "pos_lasso": pos_lasso
}, index=pd.to_datetime(oos_dates))

In [74]:
# ---------- Step 5: Statistical evaluation ----------
r2_ridge = oos_r2_vs_benchmark(results["y_true"], results["yhat_ridge"], results["y_bench_mean"])
r2_lasso = oos_r2_vs_benchmark(results["y_true"], results["yhat_lasso"], results["y_bench_mean"])

mae_ridge = mean_absolute_error(results["y_true"], results["yhat_ridge"])
mae_lasso = mean_absolute_error(results["y_true"], results["yhat_lasso"])

print("\n==============================")
print("Step 3/5: Ridge & Lasso (Walk-forward OOS + TS-CV)")
print("==============================")
print(f"OOS R^2 vs expanding-mean benchmark:")
print(f"  Ridge: {r2_ridge:.4f} ({r2_ridge*100:+.2f}%)")
print(f"  Lasso: {r2_lasso:.4f} ({r2_lasso*100:+.2f}%)")
print("\nMAE (OOS):")
print(f"  Ridge: {mae_ridge:.6f}")
print(f"  Lasso: {mae_lasso:.6f}")

print("\nAverage selected alpha (lambda):")
print(f"  Ridge: {np.mean(results['best_alpha_ridge']):.6f}")
print(f"  Lasso: {np.mean(results['best_alpha_lasso']):.6f}")

# Lasso variable selection frequency (how often selected across OOS windows)
sel_freq = (lasso_nonzero_counts / len(results)).sort_values(ascending=False)
print("\nTop Lasso selection frequencies (share of OOS windows where coef != 0):")
print(sel_freq.head(15))


Step 3/5: Ridge & Lasso (Walk-forward OOS + TS-CV)
OOS R^2 vs expanding-mean benchmark:
  Ridge: -0.0047 (-0.47%)
  Lasso: -0.0018 (-0.18%)

MAE (OOS):
  Ridge: 0.017996
  Lasso: 0.018123

Average selected alpha (lambda):
  Ridge: 4735.951188
  Lasso: 0.006175

Top Lasso selection frequencies (share of OOS windows where coef != 0):
bond_volatility        0.087248
curvature              0.060403
slope                  0.000000
short_rate_diff        0.000000
default_spread         0.000000
inflation_yoy          0.000000
unemployment           0.000000
recession_indicator    0.000000
industrial_prod_yoy    0.000000
vix_index              0.000000
bond_momentum          0.000000
y_lag1                 0.000000
dtype: float64


In [75]:
# ---------- Step 6: Economic evaluation (Ridge vs Lasso only) ----------
# Strategy excess return = position * realized excess return
# position = sign of predicted excess return

results["strat_ridge"] = results["pos_ridge"] * results["y_true"]
results["strat_lasso"] = results["pos_lasso"] * results["y_true"]

# Cumulative returns (compounded growth of $1)
results["cum_ridge"] = (1 + results["strat_ridge"]).cumprod()
results["cum_lasso"] = (1 + results["strat_lasso"]).cumprod()

# Annualized Sharpe ratios
sr_ridge = ann_sharpe(results["strat_ridge"])
sr_lasso = ann_sharpe(results["strat_lasso"])

print("\n==============================")
print("Step 6: Economic Evaluation (Ridge vs Lasso)")
print("==============================")
print(f"Annualized Sharpe:")
print(f"  Ridge timing: {sr_ridge:.3f}")
print(f"  Lasso timing: {sr_lasso:.3f}")


Step 6: Economic Evaluation (Ridge vs Lasso)
Annualized Sharpe:
  Ridge timing: 0.148
  Lasso timing: 0.211
